<a href="https://colab.research.google.com/github/SL-Ivy/RNAseq_practice/blob/main/micm_workshop_introduction_to_gwas_and_prs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1: Introduction to GWAS

In this section, we will perform the basic steps for interpreting GWAS summary statistics.

## Load GWAS summary statistics

Here, we are using the summary statistics for [coronary artery disease (CAD)](https://pubmed.ncbi.nlm.nih.gov/29212778/) from van der Harst et al. (2018). We will use these same summary statistics to compute the polygenic risk for CAD across a cohort in Part 2.

In [ ]:
# Load GWAS summary statistics
!gdown --id 1tgLX51-EShA4ZwroxTbjoNhbvsxc2V-y
!head raw_cad_gwas_sum_stats

## Count and plot the number of SNPs at different p-value thresholds

As an exercise, we want to see how many SNPs reach different p-value thresholds. We refer to genome-wide significance threshold as equal to `0.05 / # of SNPs across the genome`.

If there are approximately 1 million independent regions in the genome, then the genome-wide significant p-value threshold is equal to `0.05 / 1 million = 5e-8`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

sum_stats = pd.read_csv("raw_cad_gwas_sum_stats", delim_whitespace=True)

# Define the thresholds
thresholds = [1e-4, 1e-5, 1e-6, 5e-8]

# Count the number of SNPs below each threshold
counts = [np.sum(sum_stats['pval'] < threshold) for threshold in thresholds]

# Create the bar plot
plt.figure(figsize=(10, 6))
plt.bar([str(thresh) for thresh in thresholds], counts, color='blue')

# Add labels and title
plt.xlabel('P-value Thresholds')
plt.ylabel('Number of SNPs')
plt.title('Number of SNPs at Different P-value Thresholds')
plt.legend()

# Show the plot
plt.show()

# Part 2: Introduction to polygenic risk score

In this section, we will perform the basic steps for computing polygenic risk scores (PRSs).

## Set up command-line environment

We download PLINK because we will use it to:
* Process and QC the genotype data (.bim, .fam, .bed files).
* Compute individual-level polygenic risk scores

In [ ]:
%%capture
# Download PLINK2
!wget -qO- https://s3.amazonaws.com/plink2-assets/plink2_linux_x86_64_latest.zip > plink2.zip
!unzip -q plink2.zip
!mv plink2 /usr/local/bin/
!plink2 --version

## Load individual-level genotype data

We are using a subset of the genotyped data from the [1000 Genomes Project](https://www.internationalgenome.org/data). These data are aligned to the hg19 reference genome.

We load the data from a directory in Google Drive.

In [ ]:
!gdown 1s_5XroEhHM5uc_brlZGKZbmwPutM6Td-
!unzip EUR.zip

## Update PLINK files with mock phenotype data

We randomly assign binary phenotypes to each sample to represent whether they have a diagnosis of coronary artery disease (CAD).

In [ ]:
# Load the mock phenotype data for the samples
!gdown 1cclXWTnzkbxAoYwmiC1phihS3seDBTyp
!head CAD_phenotypes.txt

!plink2 --bfile EUR \
        --pheno CAD_phenotypes.txt \
        --make-bed \
        --out CAD_case_control

## Count the number of alleles and allele frequencies across samples

This step provides us with the allele frequency information for the variants across our samples. We perform this step to get a better sense of the distribution of allele frequencies prior to QC.

The red dashed line represents a minor allele frequency (MAF) of 0.01. This is the standard MAF cutoff that is used for QC'ing genotype data prior to computing PRS.

In [ ]:
!plink2 --bfile CAD_case_control \
        --freq \
        --out frequency_counts

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Read the .afreq file into a pandas DataFrame
afreq_df = pd.read_csv("frequency_counts.afreq", delim_whitespace=True)

# Display the first few rows of the DataFrame
print(afreq_df.head())

plt.figure(figsize=(10, 6))
plt.hist(afreq_df['ALT_FREQS'], bins=50, edgecolor='k', alpha=0.7)
plt.axvline(x=0.01, color='red', linestyle='--', linewidth=1.5, label='MAF 0.01')

plt.xlabel('Alternative Allele Frequency')
plt.ylabel('Number of Variants')
plt.title('Distribution of Alternative Allele Frequencies')
plt.grid(True)
plt.legend()
plt.show()

## QC the individual-level genotyping data

Prior to computing PRS, we want to make sure that the genotyping data only includes high-quality common variants.

The standard QC filtering criteria is as follow, based on xxx:
* ` --maf 0.01` : Filters SNPs to include only those with a minor allele frequency (MAF) above 0.01.
*` --geno 0.1 ` : Excludes SNPs with missing call rates exceeding 0.1.
* `--hwe 0.0001` : Excludes SNPs with Hardy-Weinberg equilibrium p-values below 0.0001. We test to see if the SNP is in equilibrium within the population.
* `--make-bed` : Output will be in PLINK (.bed, .bim, .fam) format


In [ ]:
# Perform initial filtering by MAF, SNP call rate, exclude specific SNPs, and keep specific samples

!plink2 --bfile CAD_case_control \
--maf 0.01 \
--geno 0.1 \
--hwe 0.0001 \
--make-bed \
--out CAD_maf-0.01_geno-0.1_hwe-1e4

# Perform LD pruning in two steps:
# Step 1: Generate list of SNPs to keep
# We perform linkage disequilibrium (LD) pruning using:
  # - a window size of 50 SNPs
  # - a step size of 5 SNPs
  # - r2 threshold of 0.2 (correlation between SNPs)
!plink2 --bfile CAD_maf-0.01_geno-0.1_hwe-1e4 \
       --indep-pairwise 50 5 0.2 \
       --out ld_pruned

# Step 2: Apply LD pruning
!plink2 --bfile CAD_maf-0.01_geno-0.1_hwe-1e4 \
       --extract ld_pruned.prune.in \
       --make-bed \
       --out CAD_EUR_QCed

## Run PRS-CS to better estimate the effect of SNPs on the trait.

Install dependencies needed for [PRS-CS](https://github.com/getian107/PRScs).

* PRS-CS is a statistical method that is used to estimate the effect sizes of SNPs based on linkage disequilibrium (LD) and the individual-level genotype data for your target sample.
* PRS-CS provides us with more accurate effect size estimates for the GWAS summary statistic SNPs.
* We perform PRS-CS before computing the individual-level polygenic risk scores.

This is an important – yet, computationally-expensive step. In this workshop, we will use the pre-computed SNP effect size estimates from PRS-CS.

To perform PRS-CS, you can follow these steps on the command line:

```
# Clone the PRS-CS repository
!git clone https://github.com/getian107/PRScs.git

# Navigate to the PRS-CS directory
%cd PRScs

# Download the pre-trained LD reference panels
!wget https://personal.broadinstitute.org/hhuang//public/PRS-CSx/Reference/1KG/ldblk_1kg_eur.tar.gz

# Extract the downloaded files
!tar -xzvf ldblk_1kg_eur.tar.gz

# Compute PRS-CS to provide accurate SNP effect sizes ahead of individual-level PRS calculation
!python PRScs.py \
        --ref_dir=/PRScs/ldblk_1kg_eur \
        --bim_prefix=[prefix of the QCed PLINK file] \
        --sst_file=[GWAS summary statistics file] \
        --n_gwas=[# of individuals included in GWAS] \
        --out_dir=[path]/[output_prefix]

# PRS-CS will output the SNP effect sizes for each chromosome separately
# To concatenate all chromosome files:
cat [output_prefix]_pst_eff_a1_b0.5_phiauto_chr*.txt | sort -n -k1 > [output_prefix]_pst_eff_a1_b0.5_phiauto_allchr.txt

# Prepare the output of PRScs so that it can be used by PLINK
# We will use this output to generate individual-level scores for our samples an allelic scoring system involving one or more SNPs.

awk '{print $2, $4, $6}' \
[output_prefix]_pst_eff_a1_b0.5_phiauto_allchr.txt | \
sed 's/ /\t/g' > [output_prefix]_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore
```

## Load the output of PRS-CS

We load the pre-computed output of PRS-CS. This file was computed using the steps outlined above and it provides us with the effect size of SNPs on CAD risk. This file is formattted to be used as an input for PLINK.

The `score` columns are:
* SNP ID
* Reference allele
* Score (numeric), which represent the effect of each SNP on CAD risk

In [ ]:
# Load PRScs output
!gdown --id 1X83iPlOSEAkrBvEwL1ab4m9FCj37I4_y # access file from Google Drive
!head CAD_EUR_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore

## Compute individual-level polygenic risk score for CAD

We use the output of PRS-CS as the `score` parameter in PLINK to give us the accurate allelic scores for the SNPs. The output of this command is an individual-level PRS for CAD (`score`).

In [ ]:
# Run PLINK2 to calculate PRS

!plink2 \
    --bfile CAD_EUR_QCed \
    --pheno CAD_phenotypes.txt \
    --score CAD_EUR_pst_eff_a1_b0.5_phiauto_allchr.PLINKscore \
    --out prs_for_cad

!head prs_for_cad.sscore

## Comparing the PRS for CAD between cases and controls

Now that we have the PRS-CAD for each sample, we can compare the difference in PRS between cases and controls.

However, first, we scale the `SCORE1_AVG` to the mean. Now, the individual-level PRS for CAD (`SCORE1_AVG_SCALED`) denotes the number of standard deviations that the person is from the average PRS of the sample. We do this to make it easier to interpret the PRS value.

Next, we perform a simple t-test to compare the mean PRS-CAD between cases and controls. If the mean PRS-CAD is significantly different, then the `p_val < 0.05`.

In our example, we find that the mean PRS-CAD in cases is `-0.0141`, and the mean PRS-CAD for controls is `0.0146`. The p-value of the t-test is ≥ 0.05, so we conclude that there is no significant difference in mean PRS-CAD between cases and controls.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from sklearn.preprocessing import StandardScaler

# Load the data into a DataFrame
file_path = "prs_for_cad.sscore"  # Update the path as needed
df = pd.read_csv(file_path, delim_whitespace=True)

# Ensure the Phenotype is binary (0 for control, 1 for case)
df['Phenotype'] = df['Phenotype'].apply(lambda x: 1 if x == 2 else 0).astype(int)

# Standardize the SCORE1_AVG values
scaler = StandardScaler()
df['SCORE1_AVG_SCALED'] = scaler.fit_transform(df[['SCORE1_AVG']])

# Split the data into cases and controls
cases = df[df['Phenotype'] == 1]['SCORE1_AVG_SCALED']
controls = df[df['Phenotype'] == 0]['SCORE1_AVG_SCALED']

# Calculate the mean SCORE1_AVG for cases and controls
mean_cases = df[df['Phenotype'] == 1]['SCORE1_AVG_SCALED'].mean()
mean_controls = df[df['Phenotype'] == 0]['SCORE1_AVG_SCALED'].mean()

print(f"Mean SCORE1_AVG for cases: {mean_cases}")
print(f"Mean SCORE1_AVG for controls: {mean_controls}")

# Perform t-test
t_stat, p_val = ttest_ind(cases, controls)
print(f"t-statistic: {t_stat}, p-value: {p_val}")

# Set the plot style
sns.set(style="white")

# Create a box plot to view the distribution of PRS
plt.figure(figsize=(10, 6))
ax = sns.boxplot(x="Phenotype", y="SCORE1_AVG_SCALED", data=df, hue="Phenotype", palette="Set2", legend=False)

# Add annotation for the p-value
x1, x2 = 0, 1  # x-coordinates for controls and cases
y, h, col = df['SCORE1_AVG_SCALED'].max() + 0.1, 0.05, 'k'
plt.plot([x1, x1, x2, x2], [y, y + h, y + h, y], lw=1.5, color=col)
plt.text((x1 + x2) * .5, y + h, f"p = {p_val:.3e}", ha='center', va='bottom', color=col)

plt.title('Distribution of scaled polygenic risk for CAD')
plt.xlabel('CAD diagnosis')
plt.ylabel('Polygenic risk for CAD')
plt.xticks([0, 1], ['Control', 'Case'])
plt.show()